In [1]:
# Predict used car price 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
%matplotlib inline

In [38]:
df = pd.read_csv('E:\\Shirin Gangal\\Python\\Python_learning\\Udemy ML\\cardekho_imputated.csv', index_col=[0])

In [39]:
df.head()

,car_name,brand,model,vehicle_age,km_driven,seller_type,fuel_type,transmission_type,mileage,engine,max_power,seats,selling_price
0,Maruti Alto,Maruti,Alto,9,120000,Individual,Petrol,Manual,19.70,796,46.30,5,120000
1,Hyundai Grand,Hyundai,Grand,5,20000,Individual,Petrol,Manual,18.90,1197,82.00,5,550000
2,Hyundai i20,Hyundai,i20,11,60000,Individual,Petrol,Manual,17.00,1197,80.00,5,215000
3,Maruti Alto,Maruti,Alto,9,37000,Individual,Petrol,Manual,20.92,998,67.10,5,226000
4,Ford Ecosport,Ford,Ecosport,6,30000,Dealer,Diesel,Manual,22.77,1498,98.59,5,570000


In [40]:
df.isnull().sum()

car_name             0
brand                0
model                0
vehicle_age          0
km_driven            0
seller_type          0
fuel_type            0
transmission_type    0
mileage              0
engine               0
max_power            0
seats                0
selling_price        0
dtype: int64

In [41]:
df.drop('car_name', axis=1, inplace= True)
df.drop('brand',axis=1, inplace= True)


In [42]:
num_features= [feature for feature in df.columns if df[feature].dtype !='O']
print('No. of Numerical features:', len(num_features))
char_features= [feature for feature in df.columns if df[feature].dtype =='O']
print('No. of Categorical features:', len(char_features))

No. of Numerical features: 7
No. of Categorical features: 4


In [43]:
print(num_features)
print(char_features)

['vehicle_age', 'km_driven', 'mileage', 'engine', 'max_power', 'seats', 'selling_price']
['model', 'seller_type', 'fuel_type', 'transmission_type']


In [44]:
discr_features= [feature for feature in num_features if len(df[feature].unique()) <=25]
print('No. of Discrete features:', len(discr_features))
print(discr_features)

No. of Discrete features: 2
['vehicle_age', 'seats']


In [45]:
df['vehicle_age'].unique()

array([ 9,  5, 11,  6,  8,  3,  2,  4,  7, 10, 14, 12,  1, 15, 13, 16, 17,
       18, 29, 19,  0, 21, 22, 25], dtype=int64)

In [46]:
cont_features= [feature for feature in num_features if feature not in discr_features]
print('No. of Continuous features:', len(cont_features))

No. of Continuous features: 5


In [47]:
from sklearn.model_selection import train_test_split
X = df.drop(['selling_price'], axis=1)
y = df['selling_price']

In [48]:
# Feature Encoding and Scaling
len(df['model'].unique())

120

In [49]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder() # assign labels based on no. of categories
X['model']= le.fit_transform(X['model'])

In [50]:
X['model']

0          7
1         54
2        118
3          7
4         38
        ... 
19537    117
19540     42
19541     77
19542    114
19543     25
Name: model, Length: 15411, dtype: int32

In [51]:
# Create column transformer with 3 types of transformers
num_features = X.select_dtypes(exclude="object").columns
onehot_columns = ['seller_type', 'fuel_type','transmission_type']
print(num_features)
print(onehot_columns)


Index(['model', 'vehicle_age', 'km_driven', 'mileage', 'engine', 'max_power',
       'seats'],
      dtype='object')
['seller_type', 'fuel_type', 'transmission_type']


In [17]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer

numeric_transform = StandardScaler()
oh_transform = OneHotEncoder(drop='first')

In [18]:
preprocessor = ColumnTransformer(
    [
        ("OneHotEncoder", oh_transform, onehot_columns),
        ("StandardScaler", numeric_transform, num_features)
    ], remainder = 'passthrough'
)

In [19]:
X=preprocessor.fit_transform(X)

In [20]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42)
X_train.shape, X_test.shape

((12328, 14), (3083, 14))

In [21]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error, accuracy_score, classification_report, ConfusionMatrixDisplay, precision_score, recall_score, f1_score,roc_auc_score
# .ensemble : RandomForest, Adaboost. linear_model: LinearRegression, Ridge, Lasso.  tree: DecisionTree

In [22]:
def evaluate_model(true, predicted):
    mae = mean_absolute_error(true, predicted)
    mse = mean_squared_error(true, predicted)
    rmse = np.sqrt(mean_squared_error(true, predicted))
    r2_square = r2_score(true, predicted)
    return mae, rmse, r2_square

In [ ]:
# from xgboost import XGBRegressor
# "Xgboost Regressor": XGBRegressor()
# xgboost_params = {"learning_rate":[0.1, 0.01], "max_depth": [5,8,12,20,30], "n_estimators": [100,200,300], 
# "colsample_bytree"" [0.5,0.8,1,0.3, 0.4]}

In [23]:
models = {  "Linear Regression": LinearRegression(),
            "Lasso": Lasso(),
            "Ridge": Ridge(),
            "Decision Tree": DecisionTreeRegressor(),
            "K-Neighbor Regressor": KNeighborsRegressor(),
            "Random Forest":RandomForestRegressor(),
            "Adaboost": AdaBoostRegressor()
         }
for i in range(len(list(models))):
    model= list(models.values())[i]
    model.fit(X_train, y_train)
    
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    
    # Evaluate Train and Test dataset
    model_train_mae , model_train_rmse, model_train_r2 = evaluate_model(y_train, y_train_pred)

    model_test_mae , model_test_rmse, model_test_r2 = evaluate_model(y_test, y_test_pred)

    
    print(list(models.keys())[i])
    
    print('Model performance for Training set')
    print("- Root Mean Squared Error: {:.4f}".format(model_train_rmse))
    print("- Mean Absolute Error: {:.4f}".format(model_train_mae))
    print("- R2 Score: {:.4f}".format(model_train_r2))

    print('----------------------------------')
    
    print('Model performance for Test set')
    print("- Root Mean Squared Error: {:.4f}".format(model_test_rmse))
    print("- Mean Absolute Error: {:.4f}".format(model_test_mae))
    print("- R2 Score: {:.4f}".format(model_test_r2))

Linear Regression
Model performance for Training set
- Root Mean Squared Error: 553855.6665
- Mean Absolute Error: 268101.6071
- R2 Score: 0.6218
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 502543.5930
- Mean Absolute Error: 279618.5794
- R2 Score: 0.6645
Lasso
Model performance for Training set
- Root Mean Squared Error: 553855.6710
- Mean Absolute Error: 268099.2226
- R2 Score: 0.6218
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 502542.6696
- Mean Absolute Error: 279614.7461
- R2 Score: 0.6645
Ridge
Model performance for Training set
- Root Mean Squared Error: 553856.3160
- Mean Absolute Error: 268059.8015
- R2 Score: 0.6218
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 502533.8230
- Mean Absolute Error: 279557.2169
- R2 Score: 0.6645
Decision Tree
Model performance for Training set
- Root Mean Squared Error: 20797.2352
- Mean Absolute Error:

In [25]:
# This shows that random forest and k nearest neighbor is the best
#Initialize few parameter for Hyperparamter tuning
knn_params = {"n_neighbors": [2, 3, 10, 20, 40, 50]}
rf_params = {"max_depth": [5, 8, 15, None, 10],
             "max_features": [5, 7, "auto", 8],
             "min_samples_split": [2, 8, 15, 20],
             "n_estimators": [100, 200, 500, 1000]}
ada_params = {"n_estimators":[50, 70, 80, 100, 120], "loss":['linear','square','exponential']}


In [26]:
# Models list for Hyperparameter tuning
randomcv_models = [('KNN', KNeighborsRegressor(), knn_params),
                   ("RF", RandomForestRegressor(), rf_params),
                   ("AdaBoost", AdaBoostRegressor(), ada_params),
                   ]

In [27]:
from sklearn.model_selection import RandomizedSearchCV

model_param = {}
for name, model, params in randomcv_models:
    random = RandomizedSearchCV(estimator=model,
                                   param_distributions=params,
                                   n_iter=100,
                                   cv=3,
                                   verbose=2,
                                   n_jobs=-1)
    random.fit(X_train, y_train)
    model_param[name] = random.best_params_

for model_name in model_param:
    print(f"---------------- Best Params for {model_name} -------------------")
    print(model_param[model_name])

Fitting 3 folds for each of 6 candidates, totalling 18 fits
Fitting 3 folds for each of 100 candidates, totalling 300 fits
Fitting 3 folds for each of 15 candidates, totalling 45 fits
---------------- Best Params for KNN -------------------
{'n_neighbors': 10}
---------------- Best Params for RF -------------------
{'n_estimators': 200, 'min_samples_split': 2, 'max_features': 5, 'max_depth': None}
---------------- Best Params for AdaBoost -------------------
{'n_estimators': 70, 'loss': 'linear'}


In [32]:
models = { "Random Forest":RandomForestRegressor(n_estimators=200, min_samples_split=2, max_features=5, max_depth= None),
          "KNN":KNeighborsRegressor(n_neighbors=10) 
         }
for i in range(len(list(models))):
    model= list(models.values())[i]
    model.fit(X_train, y_train)

    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)

    # Evaluate Train and Test dataset
    model_train_mae , model_train_rmse, model_train_r2 = evaluate_model(y_train, y_train_pred)

    model_test_mae , model_test_rmse, model_test_r2 = evaluate_model(y_test, y_test_pred)

    
    print(list(models.keys())[i])
    
    print('Model performance for Training set')
    print("- Root Mean Squared Error: {:.4f}".format(model_train_rmse))
    print("- Mean Absolute Error: {:.4f}".format(model_train_mae))
    print("- R2 Score: {:.4f}".format(model_train_r2))

    print('----------------------------------')
    
    print('Model performance for Test set')
    print("- Root Mean Squared Error: {:.4f}".format(model_test_rmse))
    print("- Mean Absolute Error: {:.4f}".format(model_test_mae))
    print("- R2 Score: {:.4f}".format(model_test_r2))

Random Forest
Model performance for Training set
- Root Mean Squared Error: 122234.9165
- Mean Absolute Error: 39138.8149
- R2 Score: 0.9816
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 208377.4786
- Mean Absolute Error: 97503.6445
- R2 Score: 0.9423
KNN
Model performance for Training set
- Root Mean Squared Error: 363460.7706
- Mean Absolute Error: 103472.0474
- R2 Score: 0.8371
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 263888.0623
- Mean Absolute Error: 117496.2131
- R2 Score: 0.9075


In [79]:
new_car = {"model":'Alto',
    "vehicle_age":2,
    "km_driven":30000,
    "mileage":19.0,
    "engine":998,
    "max_power":67,
    "seats":5,
    "seller_type":'Individual',
    "fuel_type":'Petrol',
    "transmission_type":'Automatic' 
}


In [80]:
new_car_df = pd.DataFrame([new_car])
new_car_df["model"] = le.transform(new_car_df["model"])

In [81]:
new_car_processed = preprocessor.transform(new_car_df)  # if you used ColumnTransformer

# Predict price
predicted_price = model.predict(new_car_processed)
print("Predicted Price:", predicted_price[0])

Predicted Price: 704400.0
